In [ ]:
import skimpy

In [ ]:
x = skimpy.IntArray(10, 1)

In [ ]:
x

In [ ]:
x

In [ ]:
%%time 
x[2] = 3

In [ ]:
x

In [ ]:
%%time
x[3:6] = 2

In [ ]:
x

In [ ]:
%%time
2 * x

In [ ]:
%%time
y = 2 * x

In [ ]:
y

In [ ]:
x

In [ ]:
%%time
((x + y)**x + 56 * x - 3 * x) // x**2

In [ ]:
%%time

# Build up a big array naively.
big = skimpy.IntArray(1024 * 1024, 1)
for i in range(1024):
    big[i] = i
for i in range(1024):
    big[1024 * i : 1024 * (i + 1)] = big[:1024]

In [ ]:
%%time

# Build up a big array using the builder pattern.
big = skimpy.IntArray(1024 * 1024, 1)
builder = skimpy.IntArrayBuilder(big)
for i in range(1024):
    builder[i] = i
big = builder.build()
for i in range(1, 1024):
    builder[1024 * i: 1024 * (i + 1)] = big[:1024]
big = builder.build()

In [ ]:
%%time
big * big * big * big

In [ ]:
import numpy as np

In [ ]:
%%time
x = np.zeros(1024 * 1024, dtype=np.int32)
for i in range(1024 * 1024):
    x[i] = i

In [ ]:
%%time
z = x * x * x * x